This function computes the gradient of the flow map $ \nabla \mathbf{F}_{t_0}^{t}(\mathbf{x_0}) $ at $ \mathbf{x}_0 $ at time $ t $.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| t | float | time |
| x | array (2,) | $ \mathbf{x}_0 $ |
| X | array (Ny, Nx) | X-meshgrid|
| Y | array (Ny, Nx) | Y-meshgrid|
| Interpolant | list (2,) | Interpolant[0]: Interpolant object for $ u((\mathbf{x}, t))  $ <br /> Interpolant[1]: Interpolant object for $ v((\mathbf{x}, t))  $. <br /> <br /> The interpolation, however, works only <br /> if the particle is in the defined flow domain. <br /> At the boundary of the domain, <br /> the interpolant is not defined.|
| periodic | list (2,) | periodic[0]: periodicity in x <br /> periodic[1]: periodicity in y <br />|
| defined_domain | array (Ny, Nx) | points on the meshgrid where velocity field is defined |
| bool_unsteady | bool | specifies if velocity field is unsteady/steady |
| dt_data | float | time spacing of the velocity data |
| delta | list (2,) | delta[0]: x-grid spacing of the velocity data <br /> delta[1]: y-grid spacing of the velocity data|
| aux_grid | list (2,) | aux_grid[0]: dx auxiliary spacing <br /> aux_grid[1]: dy auxiliary spacing |
| gradFmap | array (2,2) | $ \nabla \mathbf{F}_{t_0}^{t}(\mathbf{x_0}) $ |

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-1])

# add integration folder to current working path in order to access the functions
sys.path.append(parent_directory+"/integration")

In [2]:
# Import numpy
import numpy as np

# function to compute trajectory from dFdt
from ipynb.fs.defs.integration_dFdt import integration_dFdt

In [3]:
def gradient_flowmap(time, x, X, Y, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta, aux_grid):
    
    # define auxiliary grid spacing
    rho_x = aux_grid[0]
    rho_y = aux_grid[1]

    # launch trajectories from auxiliary grid
    xL = x-np.array([1, 0])*rho_x
    xLend = integration_dFdt(time, xL, X, Y, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta)[0]
    
    xR = x+np.array([1, 0])*rho_x
    xRend = integration_dFdt(time, xR, X, Y, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta)[0]

    xD = x-np.array([0, 1])*rho_y
    xDend = integration_dFdt(time, xD, X, Y, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta)[0]
        
    xU = x+np.array([0, 1])*rho_y
    xUend = integration_dFdt(time, xU, X, Y, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta)[0]
    
    gradFmap = np.zeros((2, 2, len(time)))
    
    # compute gradient of flow map over time interval
    for i in range(len(time)):
        
        gradFmap[0, 0, i] = (xRend[0, i]-xLend[0, i])/(2*rho_x)
        gradFmap[1, 0, i] = (xRend[1, i]-xLend[1, i])/(2*rho_x)
        
        gradFmap[0, 1, i] = (xUend[0, i]-xDend[0, i])/(2*rho_y)
        gradFmap[1, 1, i] = (xUend[1, i]-xDend[1, i])/(2*rho_y)
        
    return gradFmap